In [ ]:
%pip install --upgrade  langchain langchain-google-genai "langchain[docarray]" faiss-cpu pypdf langchain-community

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('Gemini_API_Key')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('Gemini_API_Key')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
model_text=ChatGoogleGenerativeAI("gemini-1.5-pro-002")

In [ ]:
model_text.invoke("Tell a story on a lost spaceship").content

In [ ]:
vision_model=ChatGoogleGenerativeAI("gemini-1.5-pro-002 ")

In [ ]:
prompt="give me summary of this image in 5 words"
image1 = "/content/house.jfif"
message= HumanMessage(
    content=[
         {
            "type": "text",
            "text": prompt,
        },
        {

            "type": "image_url",
            "image_url": image1
        }
    ]
)


In [ ]:
print(vision_model.invoke([message]).content)

In [ ]:
loader = TextLoader("/content/birds.txt")
print(loader.load()[0].page_content)

In [ ]:
text=loader.load()[0].page_content

In [ ]:
def get_text_chunks_langchain(text):
  text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=10)
  docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
  return docs

In [ ]:
docs = get_text_chunks_langchain(text)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vectorstore = FAISS.from_documents(docs,embedding=embeddings)

In [ ]:
retriever=vectorstore.as_retriever()

In [ ]:
retriever.invoke("Bird/eagle.")

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
llm_vision = ChatGoogleGenerativeAI("gemini-1.5-pro-002 ")
llm_text = ChatGoogleGenerativeAI("gemini-1.5-pro-002 ")

In [ ]:
template = """
```
{context}
```

{query}


Provide brief information and store location.
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm_text
    | StrOutputParser()
)

In [ ]:
result = rag_chain.invoke("can you give me a detail of a eagle?")

In [ ]:
display(Markdown(result))

In [ ]:
rag_chain

In [ ]:
full_chain = (
    RunnablePassthrough() | llm_vision | StrOutputParser() | rag_chain
)

full_chain

In [ ]:
image3 = "/content/eagle_cartoon.jfif"

In [ ]:
plt.imshow(image3)
plt.show()

In [ ]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Provide information on given bird and native location.",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": image3},
    ]
)

In [ ]:
result = full_chain.invoke([message])

In [ ]:
display(Markdown(result))